<img src="logo.png">

## KNN: $K$-Nearest Neighbors o $K$ vecinos más cercanos

Es un método de clasificación no paramétrico; es decir, **no requiere asumir ninguna distribución para la variable aleatoria $X=(X_1,X_2,...,X_p)$**. Este método no requiere estimar las probabilidades desconocidas $\pi_g$ de que un elemento seleccionado al azar provenga de la población $g$.

La idea es buscar, para la nueva observación que queremos clasificar, sus $K$ vecinos más cercanos, es decir, las $K$ observaciones más cercanas respecto a una medida de distancia.

El algoritmo es el siguiente:

1) Definimos una medida de distancia adecuada para las observaciones.

2) Calculamos la distancia entre la nueva observación $\boldsymbol{x_0}$ que queremos clasificar, y las observaciones que tenemos en nuestra matriz de datos.

3) Seleccionamos las $K$ observaciones más cercanas a $\boldsymbol{x_0}$, y miramos a qué grupo pertenecen.

4) Clasificamos $\boldsymbol{x_0}$ en la población a la que pertenece una mayor proporción de sus $K$ vecinos.

<img src="ml20.png">

<img src="ml21.png">

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 12]
np.random.seed(42)

# KNN - K vecinos más próximos

Vamos a ver como vamos a usar el algoritmo KNN en scikit-learn.

El algoritmo KNN se puede usar tanto en problemas de clasificación (con el estimador [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)) como en problemas de regresión (con el estimador [KNeighborsRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor))

### Cargamos los datos

Para este ejemplo vamos a usar el dataset [CSM (Conventional and Social Media Movies)](https://archive.ics.uci.edu/ml/datasets/CSM+%28Conventional+and+Social+Media+Movies%29+Dataset+2014+and+2015) que contiene información de la popularidad en redes sociales de distintas películas así como las ventas en taquilla.

In [ ]:
pelis = pd.read_csv("datos_peliculas.csv")
pelis.shape

In [ ]:
pelis.head()

Vamos a eliminar el título de las películas

In [ ]:
pelis = pelis.drop("pelicula", axis=1)

Vamos a usar este dataset para probar KNN en clasificación y en regresión

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split

### KNN para problemas de clasificación

Probamos KNN para clasificación, en concreto vamos a suponer que queremos predecir el género de una película en función de su popularidad.

In [ ]:
from sklearn.metrics import f1_score,accuracy_score

variable_objetivo_clasificacion = "genero"
variables_independientes_clasificacion = pelis.drop(
    variable_objetivo_clasificacion, axis=1).columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_clasificacion],
    pelis[variable_objetivo_clasificacion], test_size=0.20)

In [ ]:
KNeighborsClassifier?

Los parámetros más importantes a la hora de usar `KNeighborsClasifier` son:

- **n_neighbors**: El valor de K, es decir el número de vecinos que considerar a la hora de asignar una clase.
- **weights**: A la hora de votar, que importancia dar a los vecinos. Si elegimos `auto` asigna la misma importancia a todos los vecinos. Si elegimos `distance` asigna importancia a los vecinos en función de la distancia de los vecinos al punto a clasificar
- **metric**: La métrica a la hora de medir la distancia entre los puntos. Si se usa distancia de Minkowsky se puede elegir p con el parámetro `p`, que por defecto es 2 (lo que computa la distancia euclidiana).

En este caso en particular sabemos que valor elegir de K, ya que podemos asumir que el número de categorías del dataset es el total de categorías de películas del dataset de entrenamiento.

In [ ]:
k_categorias = len(y_train.unique())
k_categorias

In [ ]:
clasificador_knn = KNeighborsClassifier(n_neighbors=10, 
                                        weights="uniform")

clasificador_knn.fit(X_train, y_train)

In [ ]:
preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

Si ahora entrenamos el estimador con el argumento de pesos `weights="distance"`, vemos que funciona de forma ligeramente mejor.

In [ ]:
clasificador_knn = KNeighborsClassifier(n_neighbors=10, 
                                        weights="distance")

clasificador_knn.fit(X_train, y_train)

preds = clasificador_knn.predict(X_test)
f1_score(y_test, preds, average="micro")

Podemos usar el método `kneighbors` para devolver los k vecinos de un punto en concreto

In [ ]:
X_test.iloc[0]

In [ ]:
distancia, indice = clasificador_knn.kneighbors(
    [X_test.iloc[0]], n_neighbors=1)
distancia, indice

In [ ]:
X_train.iloc[indice[0]]

Vemos que el vecino más cercano es similar.

### KNN para problemas de regresión

Vamos a utilizar ahora el algoritmo KNN para un problema de regresión, KNN funciona igual para hacer regresiones, simplemente que en vez de una votación donde la clase más común entre los vecinos más próximos es la elegida, se hace una interpolación de los valores de la variable numérica objetivo de los vecinos.

en concreto vamos a estimar las ventas de entradas en taquilla de una película en función de su popularidad online y presupuesto.

In [ ]:
from sklearn.metrics import mean_squared_error

variable_objetivo_regresion = "ventas"
variables_independientes_regresion = pelis.drop(
    variable_objetivo_regresion, axis=1).columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pelis[variables_independientes_regresion],
    pelis[variable_objetivo_regresion], test_size=0.20)

Usamos la implementación en sklearn `KNeighborsRegressor` para problemas de regresión. Tiene los mismos hiperparámetros que `NeighborsClassifier`.

In [ ]:
KNeighborsRegressor?

In [ ]:
regresor_knn = KNeighborsRegressor(n_neighbors=10, weights="distance")

regresor_knn.fit(X_train, y_train)

In [ ]:
preds = regresor_knn.predict(X_test)
preds

In [ ]:
np.sqrt(np.abs(mean_squared_error(y_test, preds)))

Ahora vemos el funcionamiento en validación cruzada tanto del clasificador como el regresor

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
error_validacion_cruzada_clasificacion = np.sqrt(np.abs(
    cross_val_score(KNeighborsClassifier(n_neighbors=k_categorias,
                                         weights="distance"), 
                X=pelis[variables_independientes_clasificacion],
               y=pelis[variable_objetivo_clasificacion], 
               scoring="f1_micro"
        ).mean()
      )
)
print("La puntuación F1 de KNN para clasificacion en este dataset es {:.2f}".format(
    error_validacion_cruzada_clasificacion))